In [26]:
import torch
import torch.nn as nn

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, dropout=0.5):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output, hidden = self.rnn(embedded, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size),
                torch.zeros(self.n_layers, batch_size, self.hidden_size))


In [27]:
import string
char_to_num = {char: i for i, char in enumerate(string.ascii_lowercase, 1)}

# 字符到数字的映射
char_to_num = {char: i for i, char in enumerate(string.ascii_lowercase, 1)}
char_to_num['<pad>'] = 0  # 填充符
char_to_num['-'] = 27  # -
char_to_num['<eos>'] = 28  # 结束符
char_to_num['<bos>'] = 29  # 开始符（如果使用）

def tokenize(text, bidirectional=False):
    # 先全部变成小写，把大写加进来可能会出现
    lowercase_text = text.lower()
    # 27用来表示结束，如果需要可以在头上加上0，这样就可以随意给出一段文字然后来生成开始和结尾
    numbers = [char_to_num[char] for char in text if char in char_to_num] + [27]
    if bidirectional == True:
        numbers = [28] + numbers
    return numbers

In [28]:
bidirectional = False
batch_size = 64
learning_rate = 0.01
input_size = 29 + int(bidirectional)
hidden_size = 100
output_size = 29 + int(bidirectional)
n_layers = 2
dropout = 0.5
n_epochs = 100

model = RNNModel(input_size, hidden_size, output_size, n_layers, dropout)
loss_function = nn.CrossEntropyLoss(ignore_index=char_to_num['<pad>'])
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [29]:
from torch.utils.data import DataLoader
import torch.nn.functional as F
import numpy as np
from torch.nn.utils.rnn import pad_sequence

# 读取和处理数据
def load_data(filename):
    names = []
    with open(filename, 'r', encoding='utf-8') as file:
        for line in file:
            name = line.strip()
            name = tokenize(name, bidirectional=bidirectional)
            names.append(torch.tensor(name, dtype=torch.long))
    return names

f_names = load_data('female.txt')
m_names = load_data('male.txt')
dataset = f_names + m_names

lengths = [len(sequence) for sequence in dataset]
padded_dataset = pad_sequence([torch.tensor(sequence, dtype=torch.long) for sequence in dataset], \
                              batch_first=True, padding_value=0)
data_loader = DataLoader(padded_dataset, batch_size=batch_size, shuffle=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.train()
for epoch in range(n_epochs):
    for batch in data_loader:
        hidden = model.init_hidden(batch.shape[0])
        input, target = batch[:, :-1], batch[:, 1:]    # 移除最后一个字符作为输入，第二个字符到最后一个字符作为目标
        input, target = input.to(device), target.to(device)
        optimizer.zero_grad()
        output, hidden = model(input, hidden)
        hidden = tuple([each.data for each in hidden])  # 分离隐藏状态
        # 计算损失
        loss = loss_function(output.reshape(-1, output_size), target.reshape(-1))
        loss.backward()
        optimizer.step()

        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))




/var/folders/6l/gs7nyfv11wg2_50r56yjqxwm0000gn/T/ipykernel_70694/2338756709.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  padded_dataset = pad_sequence([torch.tensor(sequence, dtype=torch.long) for sequence in dataset], \


Epoch: 0/10............. Loss: 3.3828
Epoch: 0/10............. Loss: 3.2342
Epoch: 0/10............. Loss: 2.9084
Epoch: 0/10............. Loss: 2.7757
Epoch: 0/10............. Loss: 2.6728
Epoch: 0/10............. Loss: 2.6816
Epoch: 0/10............. Loss: 2.6480
Epoch: 0/10............. Loss: 2.6676
Epoch: 0/10............. Loss: 2.6523
Epoch: 0/10............. Loss: 2.5815
Epoch: 0/10............. Loss: 2.5390
Epoch: 0/10............. Loss: 2.5756
Epoch: 0/10............. Loss: 2.5208
Epoch: 0/10............. Loss: 2.4916
Epoch: 0/10............. Loss: 2.4990
Epoch: 0/10............. Loss: 2.4765
Epoch: 0/10............. Loss: 2.5433
Epoch: 0/10............. Loss: 2.4881
Epoch: 0/10............. Loss: 2.4245
Epoch: 0/10............. Loss: 2.3769
Epoch: 0/10............. Loss: 2.4775
Epoch: 0/10............. Loss: 2.4301
Epoch: 0/10............. Loss: 2.3574
Epoch: 0/10............. Loss: 2.3259
Epoch: 0/10............. Loss: 2.4044
Epoch: 0/10............. Loss: 2.4258
Epoch: 0/10.

In [30]:
torch.save(model.state_dict(), 'model_state_dict.pth')
